In [1]:
import xgboost as xg
# import dask.dataframe as dd
import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
def test_rankings(pred, order):
    for srch in pred.srch_id.unique():
        srch_pred = pred[pred.srch_id == srch]
        srch_order = order[order.srch_id == srch]
        assert len(srch_pred) == len(srch_order)
        assert (srch_pred.prop_id == srch_order.prop_id).all()

In [5]:
data = pl.read_csv('../data/raw/training_set_VU_DM.csv', null_values=['NULL'])
data.describe()

statistic,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
str,f64,str,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64
"""count""",4.958347e6,"""4958347""",4.958347e6,4.958347e6,"""251866""","""252988""",4.958347e6,4.958347e6,4.958347e6,4.950983e6,4.958347e6,4.958347e6,3.867999e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,"""317406""",3.350565e6,4.958347e6,"""119930""","""129559""","""94439""",2.024672e6,2.130269e6,556238.0,1.534288e6,1.65099e6,472797.0,"""307378""","""343663""","""131086""",2.222373e6,2.36002e6,841099.0,"""240157""","""260976""","""96174""","""315348""","""356422""","""138515""",1.916654e6,1.987503e6,614730.0,4.958347e6,138390.0,4.958347e6
"""null_count""",0.0,"""0""",0.0,0.0,"""4706481""","""4705359""",0.0,0.0,0.0,7364.0,0.0,0.0,1.090348e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""4640941""",1.607782e6,0.0,"""4838417""","""4828788""","""4863908""",2.933675e6,2.828078e6,4.402109e6,3.424059e6,3.307357e6,4.48555e6,"""4650969""","""4614684""","""4827261""",2.735974e6,2.598327e6,4.117248e6,"""4718190""","""4697371""","""4862173""","""4642999""","""4601925""","""4819832""",3.041693e6,2.970844e6,4.343617e6,0.0,4.819957e6,0.0
"""mean""",166366.561096,null,9.953133,175.340453,null,null,173.973897,70079.179496,3.180525,3.777777,0.634699,2.872589,0.130385,4.317913,16.856236,254.20959,0.21562,14042.630393,2.385427,37.474165,1.972637,0.350492,1.110525,0.502213,null,1301.234406,0.2959,null,null,null,0.04419,0.03931,18.490732,0.01729,0.057769,27.071559,null,null,null,0.022676,0.051429,29.851903,null,null,null,null,null,null,-0.060899,0.009963,22.430384,0.044749,386.283316,0.027911
"""std""",96112.230102,null,7.64689,65.916249,null,null,68.345248,40609.920378,1.051024,1.050329,0.481514,1.531011,0.159463,1.834869,10.425655,16001.237061,0.411252,8111.843351,2.053243,51.993411,0.857063,0.732569,0.417229,0.499995,null,2023.951353,0.456446,null,null,null,0.448424,0.234239,240.113851,0.457581,0.29482,1012.919911,null,null,null,0.521757,0.267836,1340.262382,null,null,null,null,null,null,0.469172,0.202914,895.965854,0.206751,821.190577,0.164716
"""min""",1.0,"""2012-11-01 00:08:29""",1.0,1.0,"""1.41""","""0.0""",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,"""-10.0001""",0.01,0.0,"""-1""","""-1""","""10""",-1.0,-1.0,2.0,-1.0,-1.0,2.0,"""-1""","""-1""","""10""",-1.0,-1.0,2.0,"""-1""","""-1""","""10""","""-1""","""-1""","""10""",-1.0,-1.0,2.0,0.0,0.0,0.0
"""25%""",82936.0,null,5.0,100.0,null,null,100.0,35010.0,3.0,3.5,0.0,1.79,0.019,4.45,8.0,85.0,0.0,7101.0,1.0,4.0,2.0,0.0,1.0,0.0,null,139.8,0.0,null,null,null,0.0,0.0,7.0,0.0,0.0,7.0,null,null,null,0.0,0.0,7.0,null,null,null,null,null,null,0.0,0.0,7.0,0.0,124.0,0.0
"""50%""",166507.0,null,5.0,219.0,null,null,219.0,69638.0,3.0,4.0,1.0,2.77,0.069,4.91,16.0,122.0,0.0,13541.0,2.0,17.0,2.0,0.0,1.0,1.0,null,386.6,0.0,null,null,null,0.0,0.0,11.0,0.0,0.0,11.0,null,null,null,0.0,0.0,12.0,null,null,null,null,null

In [6]:
data = data.sort(['srch_id', 'booking_bool', 'click_bool'], descending=[False, True, True])

In [4]:
print(data.shape)

(4958347, 54)


> CHANGE THE DATA TO ENGINEERED DATA, EVERYTHING SHOULD WORK

In [9]:
# Convert polars DataFrames to pandas DataFrames
data_pd = data.to_pandas()
# replace all NULL values with np.nan
data_pd = data_pd.replace('NULL', np.nan)
ranking_pd = data_pd[['srch_id', 'prop_id']]

# Convert object columns to appropriate data types
object_columns = data_pd.select_dtypes(include=['object']).columns
data_pd[object_columns] = data_pd[object_columns].apply(pd.to_numeric, errors='coerce')

# Split the data into features (X) and target (y)
X = data_pd.drop(['srch_id', 'prop_id'], axis=1)
y = ranking_pd['prop_id']


# Split the data into training and testing sets based on srch_id
srch_ids = data_pd['srch_id'].unique()
train_srch_ids, test_srch_ids = train_test_split(srch_ids, test_size=0.2, random_state=42)

# Create training and testing DataFrames
train_data = data_pd[data_pd['srch_id'].isin(train_srch_ids)]
test_data = data_pd[data_pd['srch_id'].isin(test_srch_ids)]

# Create training and testing ranking DataFrames
train_ranking = ranking_pd[ranking_pd['srch_id'].isin(train_srch_ids)]
test_ranking = ranking_pd[ranking_pd['srch_id'].isin(test_srch_ids)]


In [14]:


# Create XGBoost DMatrix objects for training and testing
train_dmatrix = xg.DMatrix(
    train_data.drop(['srch_id', 'booking_bool', 'gross_bookings_usd', 'position', 'click_bool'], axis=1),
    label=train_data['prop_id'],
    group=train_data['srch_id'].value_counts().sort_index().values
)
test_dmatrix = xg.DMatrix(
    test_data.drop(['srch_id', 'booking_bool', 'gross_bookings_usd', 'position', 'click_bool'], axis=1),
    label=test_data['prop_id'],
    group=test_data['srch_id'].value_counts().sort_index().values
)

# MAKE SURE TO USE FULL DATA SET IN THE END
full_dmatrix = xg.DMatrix(
    X.drop(['booking_bool', 'gross_bookings_usd', 'position', 'click_bool'], axis=1),
    label=y,
    group=data_pd['srch_id'].value_counts().sort_index().values
)

# Set XGBoost parameters
params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg',
    'learning_rate': 0.1,
    'max_depth': 7,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

# Train the XGBoost ranking model
model = xg.train(params, train_dmatrix, num_boost_round=100)



In [6]:
from sklearn.metrics import ndcg_score
# Make predictions on the test set
test_pred = model.predict(test_dmatrix)

# Evaluate the model
ndcg_scores = []
for srch_id in test_srch_ids:
    # Get the data for the current srch_id
    srch_data = test_data[test_data['srch_id'] == srch_id]
    
    # Create DMatrix for the current srch_id
    srch_dmatrix = xg.DMatrix(srch_data.drop(['srch_id', 'booking_bool', 'gross_bookings_usd', 'position', 'click_bool'], axis=1))
    
    # Make predictions for the current srch_id
    srch_pred = model.predict(srch_dmatrix)
    
    # Get the corresponding prop_ids for the current srch_id
    srch_prop_ids = srch_data['prop_id']
    
    # Sort the prop_ids based on the predicted scores
    sorted_prop_ids = srch_prop_ids.iloc[np.argsort(srch_pred)[::-1]]
    
    # Get the ground truth rankings for the current srch_id
    test_ranking_query = test_ranking[test_ranking['srch_id'] == srch_id]['prop_id']
    
    # Calculate NDCG score for the current srch_id
    ndcg = ndcg_score([test_ranking_query], [sorted_prop_ids], k=5)
    ndcg_scores.append(ndcg)

# Calculate the mean NDCG score
mean_ndcg = sum(ndcg_scores) / len(ndcg_scores)
print(f"Mean NDCG: {mean_ndcg:.4f}")

Mean NDCG: 0.4448


In [ ]:
model = xg.train(params, X, num_boost_round=100)

In [3]:
# save the model

model.save_model('model.json')
# load the model
model = xg.Booster()
model.load_model('model.json')

In [4]:

test_set = pl.read_csv('../data/raw/test_set_VU_DM.csv')
test_set = test_set.to_pandas()
test_set = test_set.replace('NULL', np.nan)

object_columns = test_set.select_dtypes(include=['object']).columns
test_set[object_columns] = test_set[object_columns].apply(pd.to_numeric, errors='coerce')

In [14]:
srch_ids = test_set['srch_id'].unique()

# Create an empty DataFrame to store the predictions
submission_df = pd.DataFrame(columns=['srch_id', 'prop_id'])

#  Unique srch_ids in the test set
srch_ids = test_set['srch_id'].unique()

# Pre-allocate a list to collect results
results = []

# Iterate over each srch_id and make predictions
for srch_id in tqdm(srch_ids):
    # Get the data for the current srch_id
    srch_data = test_set[test_set['srch_id'] == srch_id]

    # Create DMatrix for the current srch_id
    srch_dmatrix = xg.DMatrix(srch_data.drop(['srch_id'], axis=1))

    # Make predictions for the current srch_id
    srch_pred = model.predict(srch_dmatrix)

    # Get the corresponding prop_ids for the current srch_id
    srch_prop_ids = srch_data['prop_id'].values

    # Sort the prop_ids based on the predicted scores
    sorted_indices = np.argsort(srch_pred)[::-1]
    sorted_prop_ids = srch_prop_ids[sorted_indices]

    # Collect the results for the current srch_id
    results.append(pd.DataFrame({'srch_id': srch_id, 'prop_id': sorted_prop_ids}))

# Concatenate all results into a single DataFrame
submission_df = pd.concat(results, ignore_index=True)


# Check final submission DataFrame size
print(f"Expected number of entries: {len(test_set)}")
print(f"Actual number of entries: {len(submission_df)}")

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submit/submission.csv', index=False)

  0%|          | 0/199549 [00:00<?, ?it/s]

NameError: name 'test_data' is not defined